In [1]:
# import pandas as pd

# jsn = pd.read_json('./json/series.json')
# jsn = jsn.drop(columns=['valid','updatedAt', 'started', 'finished'])

import sqlite3, json
import pandas as pd

# 1. Connect to the database
conn = sqlite3.connect('../valorant_esports.db')

# 2. Use a SQL query to fetch data directly into a DataFrame
# query = "SELECT * FROM series"
query = "SELECT * FROM 'all-players'"
all_players_df = pd.read_sql_query(query, conn)

all_players_df['damageDealtTargets'] = all_players_df['damageDealtTargets'].apply(json.loads)
all_players_df['damageDealtSources'] = all_players_df['damageDealtSources'].apply(json.loads)
all_players_df['abilities'] = all_players_df['abilities'].apply(json.loads)
all_players_df['objectives'] = all_players_df['objectives'].apply(json.loads)

# 3. Close the connection
conn.close()

# all_players_df.info()

In [29]:
import sqlite3, json
import pandas as pd

# 1. Connect to the database
conn = sqlite3.connect('../valorant_esports.db')

# 2. Use a SQL query to fetch data directly into a DataFrame
# query = "SELECT * FROM series"
query = "SELECT * FROM 'all-players'"
all_players_df = pd.read_sql_query(query, conn)

all_players_df.loc[all_players_df['series_id'] == '2843061']
# query = "SELECT * FROM 'all-teams'"
# all_teams_df = pd.read_sql_query(query, conn)
# all_teams_df['series_id'].unique()
# all_players_df['series_id'].unique()

,index,series_id,round,won,side,score,team_loadoutValue,team_netWorth,team_kills,team_id,...,ultimatePoints,teamkills,alive,currentArmor,currentHealth,abilities,character.id,inventory.items,position.x,position.y


In [23]:
all_players_df['series_id'].value_counts(), all_players_df['team_id'].value_counts(),all_players_df['player_id'].value_counts()


(series_id
 2843070    40
 2843069    30
 2843063    30
 2819700    30
 2843071    30
 2843060    30
 2843062    30
 2843064    30
 2819703    30
 2819701    30
 2819699    30
 2819695    20
 2843067    20
 2843068    20
 2843061    20
 2819704    20
 2843066    20
 2819694    20
 2819698    20
 2843065    20
 2819696    20
 2819705    20
 Name: count, dtype: int64,
 team_id
 97       100
 79        75
 96        70
 1079      70
 337       60
 1611      55
 48457     30
 53367     25
 3412      25
 81        20
 281       20
 99        10
 Name: count, dtype: int64,
 player_id
 312      20
 2513     20
 297      20
 2340     20
 2358     20
          ..
 272       2
 48        2
 42        2
 206       2
 32038     2
 Name: count, Length: 61, dtype: int64)

In [7]:
common_series_ids = set(all_players_df['series_id'].unique()) & set(all_teams_df['series_id'].unique())
print(len(common_series_ids), common_series_ids )

22 {np.int64(2843060), np.int64(2843061), np.int64(2843062), np.int64(2843063), np.int64(2843064), np.int64(2843065), np.int64(2843066), np.int64(2843067), np.int64(2843068), np.int64(2843069), np.int64(2843070), np.int64(2843071), np.int64(2819694), np.int64(2819695), np.int64(2819696), np.int64(2819698), np.int64(2819699), np.int64(2819700), np.int64(2819701), np.int64(2819703), np.int64(2819704), np.int64(2819705)}


In [2]:
all_players_df['player_id'] = all_players_df['player_id'].astype(int)
all_players_df['team_id'] = all_players_df['team_id'].astype(int)

all_players_df.head(2)

,index,series_id,round,won,side,score,team_loadoutValue,team_netWorth,team_kills,team_id,...,ultimatePoints,teamkills,alive,currentArmor,currentHealth,abilities,character.id,inventory.items,position.x,position.y
0,0,2819695,1,0,attacker,13,8150,10300,96,81,...,1,0,0,0,0,"[{'abilityName': 'blast-pack', 'charges': 0}, ...",raze,[],7324.47363,-4251.37842
1,1,2819695,1,0,attacker,13,8150,10300,96,81,...,6,0,0,0,0,"[{'abilityName': 'seize', 'charges': 1}, {'abi...",fade,[],5268.81104,-3472.69482


In [3]:
all_players_df.iloc[0].to_json()

'{"index":0,"series_id":2819695,"round":1,"won":0,"side":"attacker","score":13,"team_loadoutValue":8150,"team_netWorth":10300,"team_kills":96,"team_id":81,"team_name":"MIBR (1)","player_id":14,"player_name":"aspas","participationStatus":"active","objectives":[{"id":"captureUltimateOrb","type":"captureUltimateOrb","completedFirst":true,"completionCount":5},{"id":"beginDefuseBomb","type":"beginDefuseBomb","completedFirst":false,"completionCount":2},{"id":"defuseBomb","type":"defuseBomb","completedFirst":true,"completionCount":2},{"id":"stopDefuseBomb","type":"stopDefuseBomb","completedFirst":false,"completionCount":2},{"id":"plantBomb","type":"plantBomb","completedFirst":false,"completionCount":4},{"id":"explodeBomb","type":"explodeBomb","completedFirst":false,"completionCount":1}],"money":400,"loadoutValue":1600,"netWorth":2000,"damageDealtTargets":[{"target":"body","damageAmount":1436,"occurrenceCount":55},{"target":"head","damageAmount":1280,"occurrenceCount":12},{"target":"leg","dama

In [3]:
import numpy as np
import heapq, shap

def extract_player_features(player_json):
    features = {}

    # -------------------------
    # Damage breakdown
    # -------------------------
    total_damage = player_json.get("damageDealt", 0)
    total_taken = player_json.get("damageTaken", 0)
    
    dmg_by_target = {d["target"]: d["damageAmount"] 
                     for d in player_json.get("damageDealtTargets", [])}

    features["headshot_damage_ratio"] = dmg_by_target.get("head", 0) / max(total_damage, 1)
    features["body_damage_ratio"] = dmg_by_target.get("body", 0) / max(total_damage, 1)
    features["leg_damage_ratio"] = dmg_by_target.get("leg", 0) / max(total_damage, 1)

    features["damage_efficiency"] = total_damage / max(total_taken, 1)
    features["self_damage_ratio"] = player_json.get("selfdamageDealt", 0) / max(total_damage, 1)

    # -------------------------
    # Weapon damage categories
    # -------------------------
    weapon_damage = {}
    for d in player_json.get("damageDealtSources", []):
        weapon_damage[d["target"]] = weapon_damage.get(d["target"], 0) + d["damageAmount"]

    rifle_weapons = {"vandal", "phantom", "guardian"}
    pistol_weapons = {"ghost", "sheriff", "frenzy", "classic"}
    shotgun_weapons = {"judge", "shorty"}
    ability_sources = {"paint-shells", "showstopper"}

    def ratio(weapons):
        return sum(weapon_damage.get(w, 0) for w in weapons) / max(total_damage, 1)

    features["rifle_damage_ratio"] = ratio(rifle_weapons)
    features["pistol_damage_ratio"] = ratio(pistol_weapons)
    features["shotgun_damage_ratio"] = ratio(shotgun_weapons)
    features["ability_damage_ratio"] = ratio(ability_sources)

    # -------------------------
    # Utility & survival
    # -------------------------
    features["ultimate_points"] = player_json.get("ultimatePoints", 0)
    features["alive"] = player_json.get("alive", 0)
    return features

def get_scouting_notes(player):
    notes = [] 
    if player.get('damageTaken', 0) > player.get('damageDealt', 0):
        notes.append("High-risk duelist: Takes massive damage during encounters.")
    elif player.get('damageDealt', 0) > (player.get('damageTaken', 0) * 1.5):
        notes.append("Efficient Hunter: Deals significantly more damage than he receives.")

    # Logic: Utility Handling
    if player.get('selfdamageTaken', 0) > 50:
        notes.append("Safety Concern: High self-damage recorded; check ability usage (Raze satchels/nades).")
 
    targets = player.get('damageDealtTargets', [])
    head_hits = sum(t['occurrenceCount'] for t in targets if t['target'] == 'head')
    total_hits = sum(t['occurrenceCount'] for t in targets)
    
    if total_hits > 0 and (head_hits / total_hits) > 0.25:
        notes.append("Sharp Shooter: High headshot frequency detected.")

    return notes

def build_players_ml_dataset(player_records):
    feature_rows = []
    meta_rows = [] 
    for p in player_records:
        features = extract_player_features(p)
        feature_rows.append(features) 
        player_specific_notes = get_scouting_notes(p)

        meta_rows.append({
            "player_id": p["player_id"],
            "player_name": p["player_name"],
            "series_id": p["series_id"],
            "round": p.get("round"),
            "won": False if p.get("won") == 0 else True,
            "notes" : player_specific_notes
        })

    X = pd.DataFrame(feature_rows)
    meta = pd.DataFrame(meta_rows)

    return X, meta


def build_player_predict_structure(data_json):
    X = []
    features = extract_player_features(data_json)
    X.append(features)
    return pd.DataFrame(X)

def explain_player(explainer, X_row, threshold=0.05):
    shap_output = explainer(X_row)
    shap_vals = shap_output.values[0]

    max_abs = np.max(np.abs(shap_vals))
    if max_abs == 0:
        max_abs = 1.0

    impacts = []
    for feature, value in zip(X_row.columns, shap_vals):
        impacts.append({
            "feature": feature,
            "shap_score": float(np.round(value, 3)),
            "normalized_score": float(np.round(abs(value) / max_abs, 3)),
            "direction": "positive" if value > 0 else "negative",
            "impact_level": (
                "high impact" if abs(value) > 0.15 else
                "moderate impact" if abs(value) > threshold else
                "negligible impact"
            )
        })

    strengths = [i for i in impacts if i["shap_score"] > threshold]
    weaknesses = [i for i in impacts if i["shap_score"] < -threshold]

    return strengths, weaknesses

def player_strength_score(model, X_row):
    proba = model.predict_proba(X_row)[0][1]
    return float(np.round((proba), 3))

def to_python_type(value):
    if isinstance(value, (np.integer,)):
        return int(value)
    if isinstance(value, (np.floating,)):
        return float(value)
    if isinstance(value, (np.bool_,)):
        return bool(value)
    return value

def player_win_probability(model, player_df):
    return float(np.round(model.predict_proba(player_df)[0][1],4))

def analyze_damage_targets(player_json):
    targets = player_json.get("damageDealtTargets", [])
    total_damage = player_json.get("damageDealt", 0)

    if not targets or total_damage == 0:
        return {}

    summary = []
    for t in targets:
        summary.append({
            "target": t["target"],
            "damage": t["damageAmount"],
            "hits": t["occurrenceCount"],
            "damage_ratio": np.round(t["damageAmount"] / total_damage, 3)
        })

    summary = sorted(summary, key=lambda x: x["damage"], reverse=True)

    return {
        "most_common_target": summary[0]["target"],
        "breakdown": summary
    }

def analyze_damage_sources(player_json):
    sources = player_json.get("damageDealtSources", [])
    total_damage = player_json.get("damageDealt", 0)

    if not sources or total_damage == 0:
        return {}

    summary = []
    for s in sources:
        summary.append({
            "source": s["target"],
            "damage": s["damageAmount"],
            "hits": s["occurrenceCount"],
            "damage_ratio": np.round(s["damageAmount"] / total_damage, 3)
        })

    summary = sorted(summary, key=lambda x: x["damage"], reverse=True)

    return {
        "most_common_source": summary[0]["source"],
        "breakdown": summary
    }

ECONOMY_BUCKETS = {
    "very_low": (0.0, 0.10),
    "low": (0.10, 0.18),
    "balanced": (0.18, 0.26),
    "high": (0.26, 0.35),
    "very_high": (0.35, 1.0)
}

def bucketize(value, buckets):
    for label, (low, high) in buckets.items():
        if low <= value < high:
            return label
    return "unknown"

def extract_economy_features(player_json):
    return {
        "player_loadout_ratio": float(np.round(player_json["loadoutValue"] / max(player_json["team_loadoutValue"], 1),3)),
        "player_networth_ratio": float(np.round(player_json["netWorth"] / max(player_json["team_netWorth"], 1),3)),
        "money_left_ratio": float(np.round(player_json["money"] / max(player_json["netWorth"], 1),3))
    }

def analyze_ability_suitability(player_json):
    ability_usage = {a["abilityName"]: a["charges"] for a in player_json.get("abilities", [])}

    total_ability_charges = sum(ability_usage.values()) or 1

    ability_scores = {}

    for ability, charges in ability_usage.items():
        ability_scores[ability] = {
            "usage_ratio": np.round(charges / total_ability_charges, 3),
            "usage_level": (
                "high" if charges >= 2 else
                "moderate" if charges == 1 else
                "low"
            )
        }

    best_ability = max(
        ability_scores.items(),
        key=lambda x: x[1]["usage_ratio"]
    )[0]

    return {
        "best_ability": best_ability,
        "ability_breakdown": ability_scores
    }

def analyze_objective_suitability(player_json):
    objectives = player_json.get("objectives", [])

    scores = []

    for obj in objectives:
        score = (
            obj.get("completionCount", 0) +
            (2 if obj.get("completedFirst", False) else 0)
        )

        scores.append({
            "objective": obj["type"],
            "completion_count": obj.get("completionCount", 0),
            "completed_first": obj.get("completedFirst", False),
            "suitability_score": score
        })

    scores = sorted(scores, key=lambda x: x["suitability_score"], reverse=True)

    return {
        "best_objective": scores[0]["objective"] if scores else None,
        "objective_breakdown": scores
    }

def economy_narrative(econ):
    loadout = econ["player_loadout_ratio"]
    networth = econ["player_networth_ratio"]
    money_left = econ["money_left_ratio"]

    if loadout < 0.18:
        role = "low-investment contributor"
    elif loadout < 0.26:
        role = "economy-balanced contributor"
    else:
        role = "economy-heavy carry"

    spending = (
        "conserves credits well"
        if money_left > 0.25 else
        "spends most of their credits each round"
    )

    return {
        "summary": (
            f"Player is an {role}, contributing "
            f"{int(loadout * 100)}% of team firepower. "
            f"They {spending}."
        ),
        "labels": {
            "investment_style": role,
            "credit_discipline": spending
        }
    }


AGENT_PROFILES = {
    "astra": {
        "utility_heavy": True,
        "support": True
    },
    "brimstone": {
        "utility_heavy": True,
        "support": True
    },
    "clove": {
        "utility_heavy": True,
        "support": True
    },
    "harbor": {
        "utility_heavy": True,
        "support": True
    },
    "omen": {
        "utility_heavy": True,
        "support": True
    },
    "viper": {
        "utility_heavy": True,
        "support": True
    },
    "jett": {
        "aim_heavy": True,
        "entry": True,
        "eco_friendly": False
    },
    "neon": {
        "aim_heavy": True,
        "entry": True,
        "eco_friendly": False
    },
    "phoenix": {
        "aim_heavy": True,
        "entry": True,
        "eco_friendly": False
    },
    "raze": {
        "ability_heavy": True,
        "entry": True,
        "eco_friendly": True
    },
    "reyna": {
        "aim_heavy": True,
        "entry": True,
        "eco_friendly": False
    },
    "yoru": {
        "aim_heavy": True,
        "entry": True,
        "eco_friendly": False
    },
    "iso": {
        "aim_heavy": True,
        "entry": True,
        "eco_friendly": False
    },
    "waylay": {
        "aim_heavy": True,
        "entry": True,
        "eco_friendly": False
    },
    "breach": {
        "initiator": True,
        "entry": True
    },
    "fade": {
        "initiator": True,
        "entry": True
    },
    "gekko": {
        "initiator": True,
        "entry": True
    },
    "kay/o": {
        "initiator": True,
        "support": True
    },
    "sova": {
        "initiator": True,
        "entry": True
    },
    "skye": {
        "initiator": True,
        "support": True
    },
    "tejo": {
        "initiator": True,
        "entry": True
    },
    "chamber": {
        "sentinel": True,
        "defensive": True
    },
    "cypher": {
        "sentinel": True,
        "defensive": True
    },
    "deadlock": {
        "sentinel": True,
        "defensive": True
    },
    "killjoy": {
        "sentinel": True,
        "defensive": True
    },
    "sage": {
        "sentinel": True,
        "support": True
    },
    "veto": {
        "sentinel": True,
        "defensive": True
    },
    "vyse": {
        "sentinel": True,
        "defensive": True
    }
}

def recommend_best_agent(player_features, player_json):
    economy = extract_economy_features(player_json)

    agent_scores = {}

    for agent in AGENT_PROFILES.keys():
        agent_scores[agent] = compute_agent_suitability(
            player_features,
            economy,
            agent
        )

    best_agent = max(agent_scores, key=agent_scores.get)
    top_agents = heapq.nlargest(4, agent_scores, key=agent_scores.get)

    return {
        "recommended_agent": best_agent,
        "top_agents": {agent: agent_scores[agent] for agent in top_agents},
    }


def compute_agent_suitability(player_features, economy_features, agent_id):
    profile = AGENT_PROFILES.get(agent_id, {})
    score = 0
    if profile.get("ability_heavy") and player_features["ability_damage_ratio"].values[0] > 0.25:
        score += 1

    if profile.get("entry") and player_features["headshot_damage_ratio"].values[0] > 0.3:
        score += 1

    if profile.get("eco_friendly") and economy_features["player_loadout_ratio"] < 0.25:
        score += 1

    return score

def generate_damage_target_summary(target_analysis):
    breakdown = target_analysis["breakdown"]

    body = next((x for x in breakdown if x["target"] == "body"), None)
    head = next((x for x in breakdown if x["target"] == "head"), None)
    leg = next((x for x in breakdown if x["target"] == "leg"), None)

    insights = []

    if head and head["damage_ratio"] > 0.45:
        insights.append("Relies heavily on precision aiming and headshots")

    if body and body["damage_ratio"] > 0.5:
        insights.append("Primarily deals body damage, indicating spray-based engagements")

    if leg and leg["damage_ratio"] > 0.1:
        insights.append("Frequently hits legs, suggesting rushed or unstable aim")

    if not insights:
        insights.append("Maintains balanced damage distribution across targets")

    return {
        "primary_target": target_analysis["most_common_target"],
        "summary": " | ".join(insights)
    }

RIFLES = {"phantom", "vandal", "guardian"}
SMGS = {"stinger", "spectre"}
SHOTGUNS = {"judge", "bucky"}
SNIPERS = {"operator", "marshal"}
PISTOLS = {"classic", "ghost", "sheriff", "frenzy", "shorty"}
ABILITIES = {"paint-shells", "showstopper", "boom-bot"}

def generate_weapon_summary(source_analysis):
    breakdown = source_analysis["breakdown"]

    totals = {
        "rifle": 0,
        "smg": 0,
        "shotgun": 0,
        "sniper": 0,
        "pistol": 0,
        "ability": 0
    }

    total_damage = sum(x["damage"] for x in breakdown) or 1

    for x in breakdown:
        src = x["source"]
        dmg = x["damage"]

        if src in RIFLES:
            totals["rifle"] += dmg
        elif src in SMGS:
            totals["smg"] += dmg
        elif src in SHOTGUNS:
            totals["shotgun"] += dmg
        elif src in SNIPERS:
            totals["sniper"] += dmg
        elif src in PISTOLS:
            totals["pistol"] += dmg
        elif src in ABILITIES:
            totals["ability"] += dmg

    ratios = {k: np.round(v / total_damage, 3) for k, v in totals.items()}

    insights = []

    if ratios["rifle"] > 0.6:
        insights.append("Primary rifler relying heavily on rifles")

    if ratios["ability"] > 0.25:
        insights.append("Significant damage contribution from abilities")

    if ratios["sniper"] > 0.25:
        insights.append("Sniper-oriented playstyle")

    if ratios["smg"] + ratios["shotgun"] > 0.3:
        insights.append("Prefers close-range aggressive engagements")

    if not insights:
        insights.append("Balanced weapon usage across categories")

    return {
        "most_common_weapon": source_analysis["most_common_source"],
        "weapon_damage_ratio": ratios,
        "summary": " | ".join(insights)
    }

def generate_damage_summary(target_analysis, source_analysis):
    return (
        f"Most damage dealt to {target_analysis['most_common_target']} "
        f"using {source_analysis['most_common_source']}. "
        f"Weapon usage indicates a "
        f"{'rifle-heavy' if source_analysis['most_common_source'] in RIFLES else 'mixed'} playstyle."
    )

def generate_player_report(strength_model,win_model, explainer, player_stats_df, meta , player_json):
    strengths, weaknesses = explain_player(
        strength_model, explainer, player_stats_df
    )
    
    damage_target_analysis = analyze_damage_targets(player_json)
    damage_source_analysis = analyze_damage_sources(player_json)
    economy_profile = extract_economy_features(player_json)
    eco_summary = economy_narrative(economy_profile)

    return {
        "meta": meta,
        
        "agent_fit": recommend_best_agent(player_stats_df, player_json),
        "ability_fit": analyze_ability_suitability(player_json),
        "objective_fit": analyze_objective_suitability(player_json),
        "economy_profile": extract_economy_features(player_json),
        "eco_summary" : eco_summary,
        "player_strength_score": player_strength_score(strength_model, player_stats_df),
        "strengths": strengths,
        "weaknesses": weaknesses,
        "player_win_probability": to_python_type(
            player_win_probability(win_model, player_stats_df)
        ), 
        "damage_source_analysis" : damage_source_analysis,
        "overall_damage_summary": generate_damage_summary(
            damage_target_analysis,
            damage_source_analysis
        ),
        "enemy_damage_summary": generate_damage_target_summary(damage_target_analysis),
        "damage_target_analysis": damage_target_analysis,
        "weapon_summary": generate_weapon_summary(damage_source_analysis),

        "playstyle": {
            "aim_heavy": to_python_type(player_stats_df["headshot_damage_ratio"].iloc[0] > 0.35),
            "utility_heavy": to_python_type(player_stats_df["ability_damage_ratio"].iloc[0] > 0.25),
            "rifler": to_python_type(player_stats_df["rifle_damage_ratio"].iloc[0] > 0.6)
        }
    }


/home/ndrohith/mypc/Code/GRID/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
import shap

from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


all_players_df = all_players_df.reset_index(drop=True)
data = all_players_df.to_dict(orient="records")

X, meta = build_players_ml_dataset(data)

# y = (X["damage_efficiency"] > 1.1).astype(int)
# y = (all_players_df["alive"] == 1).astype(int)

# threshold = X["damage_efficiency"].quantile(0.75)

# y = (
#     (X["damage_efficiency"] > threshold) &
#     (X["headshot_damage_ratio"] > 0.3)
# ).astype(int)

z = (X["damage_efficiency"] - X["damage_efficiency"].mean()) / X["damage_efficiency"].std()
y = (z > 0.7).astype(int)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.1, random_state=42, stratify=y
)

model = XGBClassifier(
    n_estimators=300,
    max_depth=4,
    learning_rate=0.05,
    subsample=0.9,
    colsample_bytree=0.9,
    eval_metric="logloss",
    random_state=42
)

model.fit(X_train, y_train)
preds = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, preds))

explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X_train)

y_win = all_players_df["won"].astype(int)

X_win_train, X_win_test, y_win_train, y_win_test = train_test_split(
    X, y_win, test_size=0.1, random_state=42, stratify=y_win
)

win_model = XGBClassifier(
    n_estimators=300,
    max_depth=4,
    learning_rate=0.05,
    subsample=0.9,
    colsample_bytree=0.9,
    eval_metric="logloss",
    random_state=42
)

win_model.fit(X, y_win)

win_preds = win_model.predict(X_win_test)
print("Accuracy:", accuracy_score(y_win_test, win_preds))

win_explainer = shap.TreeExplainer(win_model)
win_shap_values = win_explainer.shap_values(X_win_train)

# shap.summary_plot(shap_values, X_train)
# len(shap_values)


Accuracy: 1.0
Accuracy: 1.0


In [7]:
def get_player_df(data, player_id, round):
    df = pd.DataFrame(data)
    
    player_df = df[df['player_id'].astype(str) == str(player_id)]
    # player_df = df[(df['player_id'].astype(str) == str(player_id)) & (df['round'].astype(str) == str(round))]
    
    if player_df.empty:
        return {}, {}
    
    player_df = player_df.iloc[0]
    player_stats_df = build_player_predict_structure(player_df)
    return player_stats_df, player_df.to_dict()


def get_player_meta_json(meta, player_id):
    meta_json = meta[meta['player_id'].astype(str) == str(player_id)]
    if meta_json.empty:
        return {}
    return meta_json.iloc[0].to_dict()


# player_idx = 2
# X_row = X_test.iloc[[player_idx]]
# shap_vals = explainer.shap_values(X_row)

player_id = '14'
round = 0
player_stats_df, player_json = get_player_df(data, player_id, round)
meta_json = get_player_meta_json(meta, player_id)
# team_report = generate_team_report(model, explainer,player_df) 

player_report = generate_player_report(
    strength_model=model,
    win_model=win_model,
    explainer=explainer,
    player_stats_df=player_stats_df,
    meta=meta_json,
    player_json=player_json
    # meta.iloc[0].to_dict(),
    # scouting_notes=notes.iloc[0].to_dict()
)


player_report

TypeError: 'XGBClassifier' object is not callable

In [8]:
from collections import Counter, defaultdict
import numpy as np

def get_player_series_df(data, series_id, player_id):
    df = pd.DataFrame(data)
    
    player_df = df[(df['series_id'].astype(str) == str(series_id)) & (df['player_id'].astype(str) == str(player_id))]
    return player_df 

def aggregate_strengths_weaknesses(player_reports):
    feature_scores = defaultdict(list)

    for r in player_reports:
        for s in r.get("strengths", []):
            feature_scores[s["feature"]].append(s["shap_score"])
        for w in r.get("weaknesses", []):
            feature_scores[w["feature"]].append(w["shap_score"])

    aggregated = []
    for feature, scores in feature_scores.items():
        mean_score = np.mean(scores)
        aggregated.append({
            "feature": feature,
            "mean_shap": float(mean_score),
            "direction": "positive" if mean_score > 0 else "negative",
            "impact_level": (
                "high" if abs(mean_score) > 0.2 else
                "moderate" if abs(mean_score) > 0.1 else
                "low"
            ),
            "consistency": float(np.std(scores))
        })

    return sorted(aggregated, key=lambda x: abs(x["mean_shap"]), reverse=True)


def aggregate_win_probability(player_reports):
    probs = [r["player_win_probability"] for r in player_reports]
    return {
        "mean": float(np.mean(probs)),
        "median": float(np.median(probs)),
        "min": float(np.min(probs)),
        "max": float(np.max(probs)),
        "stability": float(1 - np.std(probs))
    }

def aggregate_win_probability(player_reports):
    probs = [r["player_win_probability"] for r in player_reports]
    return {
        "mean": float(np.mean(probs)),
        "median": float(np.median(probs)),
        "min": float(np.min(probs)),
        "max": float(np.max(probs)),
        "stability": float(1 - np.std(probs))
    }

def aggregate_economy(player_reports):
    keys = ["player_loadout_ratio", "player_networth_ratio", "money_left_ratio"]

    return {
        k: float(np.mean([r["economy_profile"][k] for r in player_reports]))
        for k in keys
    }

def aggregate_weapons(player_reports):
    weapon_counter = Counter()

    for r in player_reports:
        weapon_summary = r.get("weapon_summary", {}).get("weapon_damage_ratio", {})
        for w, v in weapon_summary.items():
            weapon_counter[w] += v

    total = sum(weapon_counter.values()) or 1

    most_common_weapon = weapon_counter.most_common(1)[0][0]

    return {
        "most_common_weapon": most_common_weapon,
        "weapon_damage_ratio": {
            w: round(v, 3)
            for w, v in weapon_summary.items()
        },
        "weapon_usage_ratio": {
            w: round(v / total, 3)
            for w, v in weapon_counter.items()
        },
        "summary": f"Primary {most_common_weapon} relying heavily on {most_common_weapon}s"
    }

def aggregate_playstyle(player_reports):
    votes = Counter()

    for r in player_reports:
        for style, active in r.get("playstyle", {}).items():
            if active:
                votes[style] += 1

    return votes.most_common(1)[0][0] if votes else "balanced"

def compute_consistency(player_reports):
    dmg = [r["player_strength_score"] for r in player_reports]
    return float(1 / (1 + np.std(dmg)))

def aggregate_player_series_reports(player_reports, player_id, series_id):
    strengths_weaknesses = aggregate_strengths_weaknesses(player_reports)
    win_prob = aggregate_win_probability(player_reports)
    economy_profile = aggregate_economy(player_reports)
    eco_summary = economy_narrative(economy_profile)
    print("player_reports" , player_reports)
    return {
        "meta": {
            "player_id": player_id,
            "series_id": series_id,
            "rounds_played": len(player_reports),
            "rounds_won": sum(1 for report in player_reports if report.get('won')),
            "rounds_lost": sum(1 for report in player_reports if not report.get('won'))
        },
        "series_strengths": [s for s in strengths_weaknesses if s["mean_shap"] > 0],
        "series_weaknesses": [s for s in strengths_weaknesses if s["mean_shap"] < 0],
        "win_probability": win_prob,
        "economy_profile": economy_profile,
        "eco_summary": eco_summary,
        "weapon_profile": aggregate_weapons(player_reports),
        "playstyle": aggregate_playstyle(player_reports),
        "payer_consistency_score": compute_consistency(player_reports)
    }


series_id = 2819695
player_id = 312

all_players_df = all_players_df.reset_index(drop=True)
data = all_players_df.to_dict(orient="records")

player_series_df = get_player_series_df(data, series_id=series_id, player_id=player_id)
player_reports = []
for _, row in player_series_df.iterrows(): 
    meta_json = get_player_meta_json(meta, row['player_id'])
    filtered_df = build_player_predict_structure(row)    
    report = generate_player_report(strength_model=model, win_model=win_model,explainer=explainer, player_stats_df=filtered_df, player_json=row.to_dict(), meta=meta_json)
    player_reports.append(report)

series_report = aggregate_player_series_reports(
    player_reports,
    player_id=player_id,
    series_id=series_id
)
series_report
# generate_player_series_report(model=model , explainer=explainer , all_players_df=data, player_id=player_id, series_id=series_id)

TypeError: 'XGBClassifier' object is not callable